<a href="https://colab.research.google.com/github/Nacho2904/CURSO-LEARNING1/blob/tareas/clasificador_de_novelas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nltk
from google.colab import drive 
import sqlite3

drive.mount('/content/gdrive')
path_a_db = 'gdrive/MyDrive/novelassql.db'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [13]:
con = sqlite3.connect(path_a_db)
cur = con.cursor()

In [17]:
cur.execute('SELECT title FROM novelas LIMIT 5')
cur.fetchall()


[('80 Years Of Signing-In At The Cold Palace, I Am Unrivalled',),
 ('999 Times Training System',),
 ('A Certain Hogwarts Magician Professor',),
 ('A Certain Marvel Super Player',),
 ('A Hundredfold Training System Instantly Upgrades 999',)]

In [22]:
df_novelas = pd.read_sql_query("SELECT * from novelas", con)

In [39]:
df_novelas.loc["id"] = pd.to_numeric(df_novelas["id"], errors = "coerce")
df_novelas = df_novelas[df_novelas["id"].notnull()]
df_novelas["id"] = np.int64(df_novelas["id"] + 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [40]:
df_novelas

,id,title,author,genres,summary,first_ten_chapters_text
0,1,Game of Thrones: Wrath of the Sleeping Dragon,Purple Taro Ice Cream,"'Action', 'Fan-Fiction'",\r\n\r\n[Song of Ice and Fire Doujin]Crossing ...,['\u3000\u3000 rumbling—\n\u3000\u3000 The dea...
1,2,A Hundredfold Training System Instantly Upgrad...,Coming for Koi,"'Action', 'Adventure', 'Fantasy', 'Xuanhuan'","\r\n\r\nTake a step, experience 10,100 times, ...","['""Ding! Congratulations on your successful bi..."
2,3,Scoring the Sacred Body of the Ancients from t...,God J,"'Action', 'Adventure', 'Fantasy', 'Xuanhuan'",\r\n\r\nJun Xiaoyao was transported to a mysti...,"['\u3000\u3000 Huangzhou, one of the three tho..."
3,4,Godly Choice System,Ye Huo Fen Xin,'Urban Life',\r\n\r\nOrdinary high school students get a go...,"['Zhonghai City, Longyang County.\nAs the top ..."
4,5,Rebirth of the Best Businesswoman at School,The cold is approaching,"'Romance', 'Urban Life'","\r\n\r\nIn my previous life, I was humiliated ...",['The warm spring sun fell sporadically on the...
...,...,...,...,...,...,...
514,515,Naruto: The Strongest Kakashi,Mo Yuan Nine Inkstones,"'Action', 'Adventure', 'Fan-Fiction', 'Fantasy'","\r\n\r\nIn the Night of Nine Tails attack, the...","['Long silver-white hair, black cloth face mas..."
515,516,Marvel Comics Devouring the God Genes,Weird,"'Fan-Fiction', 'Fantasy', 'Sci-Fi'","\r\n\r\nReborn in Marvel, Ino inherits the umb...",['Chapter 1 .Missouri: Flower of the Gods [1]\...
516,517,Uchiha’s Life Simulator,hit mango,"'Fan-Fiction', 'Fantasy', 'Xuanhuan'","\r\n\r\nPeople are in Uchiha, and there is sti...","[""Author Rong, please make a quibble. The main..."
517,518,"Sign In to Cake Island, Start With Top-Notch Haki",Pirate Merchant,'Fan-Fiction',\r\n\r\nYe Yi traveled through the world of pi...,[' New WorldTotto Land\n Totto Land is a power...
